In [1]:
import numpy as np
import os
import time

In [2]:
import tensorflow as tf

In [3]:
path_to_file = 'evgenyi_onegin.txt'
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [4]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [5]:
import re
import nltk
from string import punctuation

In [6]:
exclude = set(punctuation)

def preprocess_text(txt):
    txt = str(txt)
    #txt = re.sub("\n", " ", txt)
    txt = "".join(c for c in txt if c not in exclude)
    return "".join(txt)

In [7]:
text = preprocess_text(text)

In [8]:
text[:500]

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                                Роман в стихах\n\n                        Не мысля гордый свет забавить\n                        Вниманье дружбы возлюбя\n                        Хотел бы я тебе представить\n                        Залог достойнее тебя\n                        Достойнее души прекрасной\n                        Святой исполненной мечты\n                        Поэзии живой и ясной\n                        Высоких ду'

In [9]:
text_tokens = nltk.word_tokenize(text)

In [10]:
text_tokens[:30]

['Александр',
 'Сергеевич',
 'Пушкин',
 'Евгений',
 'Онегин',
 'Роман',
 'в',
 'стихах',
 'Не',
 'мысля',
 'гордый',
 'свет',
 'забавить',
 'Вниманье',
 'дружбы',
 'возлюбя',
 'Хотел',
 'бы',
 'я',
 'тебе',
 'представить',
 'Залог',
 'достойнее',
 'тебя',
 'Достойнее',
 'души',
 'прекрасной',
 'Святой',
 'исполненной',
 'мечты']

In [11]:
vocab = sorted(set(text_tokens))
len(vocab)

9331

In [12]:
# Creating a mapping from unique words to indices
word2idx = {u:i for i, u in enumerate(vocab)}
idx2word = np.array(vocab)

text_as_int = np.array([word2idx[w] for w in text_tokens])

In [13]:
len(text_as_int)

23058

In [14]:
len(text_tokens)

23058

In [15]:
idx2word[341]

'Везут'

In [16]:
word2idx['Везут']

341

### Train and Target

In [19]:
# The maximum length sentence you want for a single input in words
seq_length = 15
examples_per_epoch = len(text_tokens)//(seq_length+1)

# Create training examples / targets
word_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in word_dataset.take(5):
    print(idx2word[i.numpy()])

Александр
Сергеевич
Пушкин
Евгений
Онегин


In [20]:
sequences = word_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(' '.join(idx2word[item.numpy()])))

'Александр Сергеевич Пушкин Евгений Онегин Роман в стихах Не мысля гордый свет забавить Вниманье дружбы возлюбя'
'Хотел бы я тебе представить Залог достойнее тебя Достойнее души прекрасной Святой исполненной мечты Поэзии живой'
'и ясной Высоких дум и простоты Но так и быть рукой пристрастной Прими собранье пестрых глав'
'Полусмешных полупечальных Простонародных идеальных Небрежный плод моих забав Бессонниц легких вдохновений Незрелых и увядших лет Ума'
'холодных наблюдений И сердца горестных замет ГЛАВА ПЕРВАЯ И жить торопится и чувствовать спешит Кн Вяземский'


In [21]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(' '.join(idx2word[input_example.numpy()])))
    print('Target data:', repr(' '.join(idx2word[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин Евгений Онегин Роман в стихах Не мысля гордый свет забавить Вниманье дружбы'
Target data: 'Сергеевич Пушкин Евгений Онегин Роман в стихах Не мысля гордый свет забавить Вниманье дружбы возлюбя'


In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 5120

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 15), (64, 15)), types: (tf.int32, tf.int32)>

In [24]:
# Length of the vocabulary in words
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [25]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [26]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [27]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 15, 9331) # (batch_size, sequence_length, vocab_size)


In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           2388736   
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          8392704   
_________________________________________________________________
dense (Dense)                (64, None, 9331)          9564275   
Total params: 24,284,019
Trainable params: 24,284,019
Non-trainable params: 0
_________________________________________________________________


In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [32]:
example_batch_predictions[0][2]

<tf.Tensor: shape=(9331,), dtype=float32, numpy=
array([-0.00020315,  0.0014102 , -0.00129801, ...,  0.00037309,
       -0.00055037, -0.00010976], dtype=float32)>

In [34]:
print("Input: \n", repr("".join(idx2word[input_example_batch[0][2]])))
print("Next Char Predictions: \n", repr(" ".join(idx2word[sampled_indices])))

Input: 
 'проворно'
Next Char Predictions: 
 'полураскрытый Нести хижина такихто прадедов пей сказки Свершить домашним наводят одинокий старины пламень боком опасной'


### Train the model

In [35]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 15, 9331)  # (batch_size, sequence_length, vocab_size)
Scalar_loss:       9.141065


In [36]:
model.compile(optimizer='adam', loss=loss)

In [37]:
# Configure checkpoints
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=20,
    save_weights_only=True)

In [38]:
EPOCHS = 200

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 22 steps
Epoch 1/200
22/22 [==============================] - 2s 85ms/step - loss: 8.7340
Epoch 2/200
22/22 [==============================] - 1s 36ms/step - loss: 8.2756
Epoch 3/200
22/22 [==============================] - 1s 36ms/step - loss: 8.0739
Epoch 4/200
22/22 [==============================] - 1s 36ms/step - loss: 7.9093
Epoch 5/200
22/22 [==============================] - 1s 36ms/step - loss: 7.7840
Epoch 6/200
22/22 [==============================] - 1s 36ms/step - loss: 7.6953
Epoch 7/200
22/22 [==============================] - 1s 36ms/step - loss: 7.6164
Epoch 8/200
22/22 [==============================] - 1s 36ms/step - loss: 7.5033
Epoch 9/200
22/22 [==============================] - 1s 36ms/step - loss: 7.3551
Epoch 10/200
22/22 [==============================] - 1s 36ms/step - loss: 7.1999
Epoch 11/200
22/22 [==============================] - 1s 36ms/step - loss: 7.0407
Epoch 12/200
22/22 [==============================] - 1s 35ms/step - loss: 6.8660
Epoch 

22/22 [==============================] - 1s 59ms/step - loss: 0.2285
Epoch 101/200
22/22 [==============================] - 1s 35ms/step - loss: 0.2221
Epoch 102/200
22/22 [==============================] - 1s 34ms/step - loss: 0.2127
Epoch 103/200
22/22 [==============================] - 1s 35ms/step - loss: 0.2085
Epoch 104/200
22/22 [==============================] - 1s 35ms/step - loss: 0.2069
Epoch 105/200
22/22 [==============================] - 1s 34ms/step - loss: 0.2256
Epoch 106/200
22/22 [==============================] - 1s 34ms/step - loss: 0.2311
Epoch 107/200
22/22 [==============================] - 1s 35ms/step - loss: 0.2221
Epoch 108/200
22/22 [==============================] - 1s 35ms/step - loss: 0.2105
Epoch 109/200
22/22 [==============================] - 1s 35ms/step - loss: 0.1984
Epoch 110/200
22/22 [==============================] - 1s 35ms/step - loss: 0.1899
Epoch 111/200
22/22 [==============================] - 1s 35ms/step - loss: 0.1836
Epoch 112/200
22/2

22/22 [==============================] - 1s 37ms/step - loss: 0.0851
Epoch 199/200
22/22 [==============================] - 1s 37ms/step - loss: 0.0846
Epoch 200/200
22/22 [==============================] - 1s 61ms/step - loss: 0.0863


In [39]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_200'

In [40]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [41]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            2388736   
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
dense_1 (Dense)              (1, None, 9331)           9564275   
Total params: 24,284,019
Trainable params: 24,284,019
Non-trainable params: 0
_________________________________________________________________


In [44]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 16

    # Converting our start string to numbers (vectorizing)
    input_eval = [word2idx[s] for s in nltk.word_tokenize(start_string)]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2word[predicted_id])

    return (start_string +' '+ ' '.join(text_generated))

In [57]:
print(generate_text(model, start_string=u"Грозы"))

Грозы сады монастыри Бухарцы сани огороды Купцы лачужки мужики Бульвары башни казаки Аптеки магазины моды Балконы XLVI


In [58]:
print(generate_text(model, start_string=u"Залог"))

Залог пред камином стол накрыт Евгений ждет вот едет Ленский На тройке чалых лошадей Давай обедать поскорей


In [69]:
print(generate_text(model, start_string=u"И все"))

И все ей кажется бесценным Все душу томную живит Полумучительной отрадой И стол с померкшею лампадой Без элегических
